In [39]:
import random
import math

class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0.0 #starting off the gradient with 0
        self._backward = lambda: None # a placeholder for the function that computes the local gradient
        self._prev = set(_children)#storing all the elements needed for back propagation
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data:.4f})"

    def __add__(self, other): # Defines the backward pass (gradient calculation) for addition
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+') 
        def _backward():
            self.grad += out.grad 
            other.grad += out.grad
        out._backward = _backward
        return out

    def __mul__(self, other): # Defines the backward pass (gradient calculation) for multiplication
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other): # Defines the backward pass (gradient calculation) for powers
        assert isinstance(other, (int, float))
        out = Value(self.data**other, (self,), f'**{other}')
        def _backward():
            self.grad += (other * (self.data**(other - 1))) * out.grad
        out._backward = _backward
        return out
    
    def relu(self): #using relu for activation(considers everything below 0 to be 0)
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')
        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward
        return out

    def tanh(self): # Defines the backward pass (gradient calculation) for tanh
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        # Topologically sort all nodes in the computation graph
        self.grad = 1.0
        for node in reversed(topo):
            node._backward() #backpropagating through each and every node
            

    # some more additional methods
    def __radd__(self, other):
        return self + other

    def __rmul__(self, other):
        return self * other

    def __neg__(self): 
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __truediv__(self, other): 
        return self * other**-1




In [41]:
class Neuron:
    # building a neuron
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)] #initializing weights randomly
        self.b = Value(random.uniform(-1, 1)) #initializing bias randomly

    def __call__(self, x):
        # Calculate activation: w * x + b
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    """A layer of neurons."""
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    """A Multi-Layer Perceptron (a simple neural network)."""
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [47]:
#testing how it works
model = MLP(2, [4, 4, 1]) 

xs = [
  [2.0, 3.0],
  [-1.0, 2.5],
  [1.0, 1.0],
  [1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0] 
print("--- Training Start ---")
for k in range(20):
    # a small training to see how weights and biases adapt
    ypred = [model(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    for p in model.parameters():
        p.grad = 0.0
    loss.backward()

    learning_rate = 0.05
    for p in model.parameters():
        p.data += -learning_rate * p.grad
    
    print(f"Step {k+1}: loss {loss.data:.4f}")

print("\n--- Training Finished ---")

print("\nPredictions on the training data:")
final_predictions = [model(x) for x in xs]
for (x_val, y_gt, y_pred) in zip(xs, ys, final_predictions):
    print(f"Input: {x_val}, Target: {y_gt}, Prediction: {y_pred.data:.4f}")

--- Training Start ---
Step 1: loss 3.7279
Step 2: loss 3.3675
Step 3: loss 3.0725
Step 4: loss 2.8205
Step 5: loss 2.5923
Step 6: loss 2.3759
Step 7: loss 2.1651
Step 8: loss 1.9585
Step 9: loss 1.7574
Step 10: loss 1.5640
Step 11: loss 1.3806
Step 12: loss 1.2096
Step 13: loss 1.0532
Step 14: loss 0.9131
Step 15: loss 0.7900
Step 16: loss 0.6836
Step 17: loss 0.5927
Step 18: loss 0.5158
Step 19: loss 0.4511
Step 20: loss 0.3967

--- Training Finished ---

Predictions on the training data:
Input: [2.0, 3.0], Target: 1.0, Prediction: 0.6224
Input: [-1.0, 2.5], Target: -1.0, Prediction: -0.9284
Input: [1.0, 1.0], Target: -1.0, Prediction: -0.6061
Input: [1.0, -1.0], Target: 1.0, Prediction: 0.7809
